### Librerias


In [40]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm

In [2]:
original = pd.read_csv('insurance.csv')
df = original.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [5]:
df["sex"].value_counts()

sex
male      676
female    662
Name: count, dtype: int64

In [6]:
df["children"].value_counts()

children
0    574
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64

In [7]:
df["smoker"].value_counts()

smoker
no     1064
yes     274
Name: count, dtype: int64

In [8]:
df["region"].value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [9]:
df[df.duplicated()]

,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.5631


In [10]:
df.shape

(1338, 7)

In [11]:
df.drop_duplicates(inplace = True)

In [12]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [13]:
# Tu código aquí debajo
def outliers(d,v):
    import numpy as np
    q3=np.quantile(d[v],0.75)
    q1=np.quantile(d[v],0.25)
    ric=q3-q1
    df_outliers=d.loc[(d[v]>=q3+1.5*ric) | (d[v]<=q1-1.5*ric), : ]
    df_limpio= d.loc[(d[v]<q3+1.5*ric) & (d[v]>q1-1.5*ric), : ]
    return df_outliers,df_limpio

In [14]:
# NO EJECUTAR
#num = ["age","bmi","children"]
#for i in num:
#    a,b=outliers(df,i)
#    df = b

In [15]:
def ident_cat_num(df): 
    # ESTA FUNCION IDENTIFICA SI LOS DATOS SON DE TIPO NUMERICO O CATEGORICO
    import pandas as pd
    #CREAMOS UNA LISTA PARA DESPUÉS ALMACENAR LOS NOMBRES DE LAS COLUMNAS QUE SEAN NUMERICAS
    list_numericas = [] 
    #CREAMOS UNA LISTA PARA DESPUÉS ALMACENAR LOS NOMBRES DE LAS COLUMNAS QUE SEAN CATEGORICAS
    list_categoricas = []
    #RECORREMOS LAS COLUMNAS DE LA BASE DE DATOS
    for i in df.columns: 
        #IDENTIFICAMOS SI LOS DATOS QUE CONTIENE LA COLUMNA SON NUMERICOS
        if pd.api.types.is_numeric_dtype(df[i])== True:
            #ALMACENAMOS LOS NOMBRES DE LAS COLUMNAS QUE SEAN NUM
            list_numericas.append(i)
        else:
            #SINO LOS ALMACENAMOS LOS NOMBRES DE LAS COLUMNAS QUE SEAN CATEGORICAS
            list_categoricas.append(i)
    return list_numericas,list_categoricas

In [16]:
ident_cat_num(df)

(['age', 'bmi', 'children', 'charges'], ['sex', 'smoker', 'region'])

### One hot encoding

In [17]:
onehot=OneHotEncoder()

a=onehot.fit_transform(df[['sex','smoker','region']])

encoded_df = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(['sex','smoker','region']))

df.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

df=pd.concat([df,encoded_df],axis=1)

df.drop(columns=['sex','smoker','region'],inplace=True)

print(df.shape)
df.sample(5)

(1337, 12)


,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
281,54,40.565,3,48549.17835,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1288,44,34.320,1,7147.47280,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1241,22,21.280,3,4296.27120,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
632,40,22.705,2,7173.35995,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
813,36,34.430,2,5584.30570,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [18]:
X = df.drop(columns='charges',inplace = False)
y = df['charges']
print("X",X.shape)
print("y",y.shape)

X (1337, 11)
y (1337,)


## Evaluamos modelo 1 ( LINEAR REGRESSION)

In [19]:
kfold = KFold(n_splits=10)

estandarizador = StandardScaler()
LR = LinearRegression()
media_errores= []

for i_train, i_test in kfold.split(df):
    X_train, X_test = X.iloc[i_train], X.iloc[i_test]
    y_train, y_test = y.iloc[i_train], y.iloc[i_test]
    estandarizador.fit(X_train)
    X_train_std = estandarizador.transform(X_train)
    X_test_std = estandarizador.transform(X_test)
    
    LR.fit(X_train_std,y_train)
    predictions = LR.predict(X_test_std)
    
    media_errores.append(mean_squared_error(y_test, predictions))

In [20]:
MSE = sum(media_errores) / len(media_errores)
RMSE = MSE**(1/2)

print("MSE: ",MSE)
print("RMSE: ",RMSE)

MSE:  37097284.058616266
RMSE:  6090.753981127154


## Modelo final 1

- Con todos los datos, no separamos train y test

In [21]:
X_std = estandarizador.fit_transform(X)

In [22]:
# Entrenamos con todos los datos X_std / y
LR.fit(X_std,y)
# Este modelo ya está entrenado y puede predecir

LinearRegression()

In [23]:
predicciones = LR.predict(X_std)
predicciones

array([26315.56240675,  3035.56240675,  6107.56240675, ...,
        4635.56240675,  2059.56240675, 37851.56240675])

In [24]:
df['Predicciones'] = predicciones

In [29]:
df[["charges","Predicciones"]].sample(10)

,charges,Predicciones
82,37165.16380,29275.562407
769,27941.28758,15947.562407
516,27724.28875,4059.562407
84,39836.51900,34059.562407
96,12105.32000,13323.562407
391,2138.07070,5979.562407
1005,4438.26340,5019.562407
166,4830.63000,7627.562407
113,2404.73380,5915.562407
792,21195.81800,31067.562407


## Evaluamos modelo 2 ( RANDOM FOREST REGRESSOR)

In [35]:
estandarizador = StandardScaler()
RF = RandomForestRegressor(n_estimators = 100)
media_errores= []

for i_train, i_test in kfold.split(df):
    X_train, X_test = X.iloc[i_train], X.iloc[i_test]
    y_train, y_test = y.iloc[i_train], y.iloc[i_test]
    estandarizador.fit(X_train)
    X_train_std = estandarizador.transform(X_train)
    X_test_std = estandarizador.transform(X_test)
    
    RF.fit(X_train_std,y_train)
    predictions = RF.predict(X_test_std)
    
    media_errores.append(mean_squared_error(y_test, predictions))

In [36]:
MSE = sum(media_errores) / len(media_errores)
RMSE = MSE**(1/2)

print("MSE: ",MSE)
print("RMSE: ",RMSE)

MSE:  23621687.936383247
RMSE:  4860.214803522911


## Modelo final 2 

In [37]:
RF.fit(X_std,y)
predicciones = RF.predict(X_std)
df['Predicciones_RF'] = predicciones

In [38]:
df[["charges","Predicciones_RF"]].sample(10)

,charges,Predicciones_RF
673,46200.98510,45926.952378
223,34779.61500,34957.777171
652,8527.53200,8681.051152
1000,34472.84100,34665.315232
996,13887.96850,14016.082320
1090,11286.53870,11832.137241
992,5484.46730,5651.268770
201,8871.15170,8838.975976
1231,12479.70895,12703.507359
487,1253.93600,1275.846437


## Evaluamos modelo 3 ( SUPPORT VECTOR MACHINE )

In [41]:
kfold = KFold(n_splits=10)

estandarizador = StandardScaler()
SVM = svm.SVR()
media_errores= []

for i_train, i_test in kfold.split(df):
    X_train, X_test = X.iloc[i_train], X.iloc[i_test]
    y_train, y_test = y.iloc[i_train], y.iloc[i_test]
    estandarizador.fit(X_train)
    X_train_std = estandarizador.transform(X_train)
    X_test_std = estandarizador.transform(X_test)
    
    SVM.fit(X_train_std,y_train)
    predictions = SVM.predict(X_test_std)
    
    media_errores.append(mean_squared_error(y_test, predictions))

In [42]:
MSE = sum(media_errores) / len(media_errores)
RMSE = MSE**(1/2)

print("MSE: ",MSE)
print("RMSE: ",RMSE)

MSE:  161030072.65919384
RMSE:  12689.762513900481


## Modelo final 3

In [43]:
SVM.fit(X_std,y)
predicciones = SVM.predict(X_std)
df['Predicciones_SVM'] = predicciones

In [45]:
df[["charges","Predicciones_SVM"]].sample(10)

,charges,Predicciones_SVM
1000,34472.84100,9459.070648
1260,3277.16100,9340.929957
233,12333.82800,9400.753944
271,42856.83800,9477.777741
867,13129.60345,9417.628311
348,5478.03680,9338.103935
12,1826.84300,9333.592235
810,6360.99360,9384.169645
1204,5116.50040,9352.545096
1280,24535.69855,9471.596072
